In [203]:
using LowLevelFEM, LinearAlgebra, SparseArrays
using Tensors
using StaticArrays

In [204]:
function materialTangentMatrix0(
    problem::Problem;
    F::TensorField,
    C::Union{AbstractMatrix,Nothing}=nothing,
    energy::Union{Nothing,Function}=nothing,
    params=nothing)

    @assert xor(C !== nothing, energy !== nothing)
    @assert problem.dim == 3 && problem.pdim == 3

    gmsh.model.setCurrent(problem.name)

    pdim = 3
    dof = pdim * problem.non

    # --- elementwise nodal F
    Fe = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    # --- preprocess constant / field C
    Centry = nothing
    if C !== nothing
        Centry = Matrix{Any}(undef, 6, 6)
        for I in CartesianIndices(C)
            cij = C[I]
            Centry[I] = cij isa ScalarField ?
                        Dict(zip(nodesToElements(cij).numElem,
                nodesToElements(cij).A)) :
                        Float64(cij)
        end
    end

    len = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Int[]
    J = Int[]
    V = Float64[]
    sizehint!(I, len)
    sizehint!(J, len)
    sizehint!(V, len)

    for mat in problem.material
        for (_, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(problem.dim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss$(2order+1)")

                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                # --- reusable buffers
                B = zeros(6, 3numNodes)
                Ke = zeros(3numNodes, 3numNodes)
                Cgp = zeros(6, 6)
                Fgp = zeros(3, 3)

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]
                    Fnode = Fmap[elem]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    fill!(Ke, 0.0)

                    for k in eachindex(wip)
                        invJ = inv(Jac[:, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- F at Gauss
                        fill!(Fgp, 0.0)
                        @inbounds for a in 1:numNodes
                            Na = H[a, k]
                            Fa = @view Fnode[9a-8:9a, 1]
                            Fgp .+= Na .* reshape(Fa, 3, 3)
                        end

                        if energy !== nothing
                            # --- energy path
                            Fm = SMatrix{3,3}(Fgp)
                            Cmat = transpose(Fm) * Fm
                            Ctan = LowLevelFEM.tangent_from_energy(energy, Cmat, params)
                            Cgp .= Ctan
                        else
                            # --- old path
                            @inbounds for i in 1:6, j in 1:6
                                cij = Centry[i, j]
                                Cgp[i, j] = cij isa Float64 ?
                                            cij :
                                            dot(cij[elem][:, 1], H[:, k])
                            end
                        end

                        # --- build B
                        fill!(B, 0.0)
                        @inbounds for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a, k]
                            for j in 1:3
                                v = @view Fgp[j, :]
                                col = (a - 1) * 3 + j
                                B[1, col] = v[1] * ∇Na[1]
                                B[2, col] = v[2] * ∇Na[2]
                                B[3, col] = v[3] * ∇Na[3]
                                B[4, col] = v[2] * ∇Na[3] + v[3] * ∇Na[2]
                                B[5, col] = v[1] * ∇Na[3] + v[3] * ∇Na[1]
                                B[6, col] = v[1] * ∇Na[2] + v[2] * ∇Na[1]
                            end
                        end

                        Ke .+= (B' * Cgp * B) * w
                    end

                    # --- scatter
                    @inbounds for a in 1:3numNodes, b in 1:3numNodes
                        Ia = (nodeTags[div(a - 1, 3)+1] - 1) * 3 + mod(a - 1, 3) + 1
                        Ib = (nodeTags[div(b - 1, 3)+1] - 1) * 3 + mod(b - 1, 3) + 1
                        push!(I, Ia)
                        push!(J, Ib)
                        push!(V, Ke[a, b])
                    end
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


materialTangentMatrix0 (generic function with 1 method)

In [205]:
function materialTangentMatrix1(
    problem::Problem;
    F::TensorField,
    C::Union{AbstractMatrix,Nothing}=nothing,
    energy::Union{Nothing,Function}=nothing,
    params=nothing
)
    @assert xor(C !== nothing, energy !== nothing)
    @assert problem.dim == 3 && problem.pdim == 3

    gmsh.model.setCurrent(problem.name)

    pdim = 3
    dof = pdim * problem.non

    # --- elementwise nodal F
    Fe = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    # --- preprocess constant / field C (old path)
    Centry = nothing
    if C !== nothing
        Centry = Matrix{Any}(undef, 6, 6)
        for Icart in CartesianIndices(C)
            cij = C[Icart]
            if cij isa ScalarField
                ce = nodesToElements(cij)
                Centry[Icart] = Dict(zip(ce.numElem, ce.A)) # elem => nodal array
            else
                Centry[Icart] = Float64(cij)
            end
        end
    end

    # --- prealloc sparse triplets (pos-style, no push!)
    len = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, len)
    J = Vector{Int}(undef, len)
    V = Vector{Float64}(undef, len)
    pos = 1

    for mat in problem.material
        for (_, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(problem.dim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss$(2order+1)")

                # shape functions
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # gradients
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                ndofe = 3 * numNodes

                # --- reusable buffers (element-level)
                B = zeros(6, ndofe)
                Ke = zeros(ndofe, ndofe)
                Cgp = zeros(6, 6)
                Fgp = zeros(3, 3)

                # buffers for Ke update: tmp6 = Cgp*B
                tmp6 = zeros(6, ndofe)

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]
                    Fnode = Fmap[elem]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    fill!(Ke, 0.0)

                    for k in eachindex(wip)
                        # --- invJ via StaticArrays (fast, no heap)
                        Jm = @SMatrix [
                            Jac[1, 3k-2] Jac[1, 3k-1] Jac[1, 3k];
                            Jac[2, 3k-2] Jac[2, 3k-1] Jac[2, 3k];
                            Jac[3, 3k-2] Jac[3, 3k-1] Jac[3, 3k]
                        ]
                        invJ = transpose(inv(Jm))
                        w = jacDet[k] * wip[k]

                        # --- F at Gauss (avoid reshape alloc by manual fill)
                        fill!(Fgp, 0.0)
                        @inbounds for a in 1:numNodes
                            Na = H[a, k]
                            base = 9a - 8

                            Fgp[1, 1] += Na * Fnode[base+0, 1]
                            Fgp[2, 1] += Na * Fnode[base+1, 1]
                            Fgp[3, 1] += Na * Fnode[base+2, 1]

                            Fgp[1, 2] += Na * Fnode[base+3, 1]
                            Fgp[2, 2] += Na * Fnode[base+4, 1]
                            Fgp[3, 2] += Na * Fnode[base+5, 1]

                            Fgp[1, 3] += Na * Fnode[base+6, 1]
                            Fgp[2, 3] += Na * Fnode[base+7, 1]
                            Fgp[3, 3] += Na * Fnode[base+8, 1]
                        end

                        # --- C at Gauss
                        if energy !== nothing
                            Fm = SMatrix{3,3,Float64}(Fgp)
                            Cmat = transpose(Fm) * Fm
                            Ctan = LowLevelFEM.tangent_from_energy(energy, Cmat, params)  # expected 6×6
                            Cgp .= Ctan
                        else
                            @inbounds for i in 1:6, j in 1:6
                                cij = Centry[i, j]
                                if cij isa Float64
                                    Cgp[i, j] = cij
                                else
                                    nod = cij[elem][:, 1]
                                    Cgp[i, j] = dot(nod, @view H[:, k])
                                end
                            end
                        end

                        # --- build B (no allocations in ∇Na, v)
                        fill!(B, 0.0)
                        @inbounds for a in 1:numNodes
                            gh = @view ∇h[3a-2:3a, k]
                            ∇Na1 = invJ[1, 1] * gh[1] + invJ[1, 2] * gh[2] + invJ[1, 3] * gh[3]
                            ∇Na2 = invJ[2, 1] * gh[1] + invJ[2, 2] * gh[2] + invJ[2, 3] * gh[3]
                            ∇Na3 = invJ[3, 1] * gh[1] + invJ[3, 2] * gh[2] + invJ[3, 3] * gh[3]

                            for j in 1:3
                                v1 = Fgp[j, 1]
                                v2 = Fgp[j, 2]
                                v3 = Fgp[j, 3]
                                col = (a - 1) * 3 + j

                                B[1, col] = v1 * ∇Na1
                                B[2, col] = v2 * ∇Na2
                                B[3, col] = v3 * ∇Na3

                                # engineering shear rows (E23,E13,E12) already with factor 2 in B definition
                                B[4, col] = v2 * ∇Na3 + v3 * ∇Na2
                                B[5, col] = v1 * ∇Na3 + v3 * ∇Na1
                                B[6, col] = v1 * ∇Na2 + v2 * ∇Na1
                            end
                        end

                        # --- Ke += w * (B' * (Cgp*B)) using mul!
                        mul!(tmp6, Cgp, B)                    # tmp6 = Cgp*B
                        mul!(Ke, transpose(B), tmp6, w, 1.0)  # Ke = w*B'*tmp6 + 1*Ke
                    end

                    # --- scatter (pos-style)
                    @inbounds for a in 1:ndofe, b in 1:ndofe
                        I[pos] = (nodeTags[div(a - 1, 3)+1] - 1) * 3 + mod(a - 1, 3) + 1
                        J[pos] = (nodeTags[div(b - 1, 3)+1] - 1) * 3 + mod(b - 1, 3) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


materialTangentMatrix1 (generic function with 1 method)

In [206]:
function initialStressMatrix0(
    problem::Problem;
    stress::Union{TensorField,Nothing}=nothing,
    energy::Union{Nothing,Function}=nothing,
    params=nothing,
    C::Union{Nothing,TensorField}=nothing)

    @assert xor(stress !== nothing, energy !== nothing)
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = dim
    dof = pdim * problem.non

    if stress !== nothing
        Se = nodesToElements(stress)
        Smap = Dict(zip(Se.numElem, Se.A))
    else
        Ce = nodesToElements(C)
        Cmap = Dict(zip(Ce.numElem, Ce.A))
    end

    len = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Int[]
    J = Int[]
    V = Float64[]
    sizehint!(I, len)
    sizehint!(J, len)
    sizehint!(V, len)

    for mat in problem.material
        for (_, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(dim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss$(2order+1)")

                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                Ke = zeros(pdim * numNodes, pdim * numNodes)
                Sgp = zeros(dim, dim)

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    fill!(Ke, 0.0)

                    for k in eachindex(wip)
                        invJ = inv(Jac[:, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        if stress !== nothing
                            Selem = Smap[elem]
                            fill!(Sgp, 0.0)
                            @inbounds for a in 1:numNodes
                                Sgp .+= H[a, k] .* reshape(Selem[9a-8:9a, 1], dim, dim)
                            end
                        else
                            Celem = Cmap[elem]
                            Cgp = @SMatrix zeros(dim, dim)
                            @inbounds for a in 1:numNodes
                                Cgp += H[a, k] .* reshape(Celem[9a-8:9a, 1], dim, dim)
                            end
                            Sgp .= LowLevelFEM.stress_from_energy(energy, Cgp, params)
                        end

                        @inbounds for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]
                            g = dot(∇Na, Sgp * ∇Nb)
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + i
                                Ke[ia, ib] += g * w
                            end
                        end
                    end

                    @inbounds for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        Ia = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        Ib = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        push!(I, Ia)
                        push!(J, Ib)
                        push!(V, Ke[a, b])
                    end
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


initialStressMatrix0 (generic function with 1 method)

In [207]:
function initialStressMatrix1(
    problem::Problem;
    stress::Union{TensorField,Nothing}=nothing,
    energy::Union{Nothing,Function}=nothing,
    params=nothing,
    C::Union{Nothing,TensorField}=nothing)

    @assert xor(stress !== nothing, energy !== nothing)
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = dim
    dof = pdim * problem.non

    # --- elementwise maps
    Smap = nothing
    Cmap = nothing
    if stress !== nothing
        Se = nodesToElements(stress)
        Smap = Dict(zip(Se.numElem, Se.A))
    else
        Ce = nodesToElements(C)
        Cmap = Dict(zip(Ce.numElem, Ce.A))
    end

    # --- prealloc sparse triplets (pos-style, no push!)
    len = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, len)
    J = Vector{Int}(undef, len)
    V = Vector{Float64}(undef, len)
    pos = 1

    for mat in problem.material
        for (_, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(dim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss$(2order+1)")

                # shape funcs
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # grad shape funcs
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                # reusable buffers (element-level)
                Ke = zeros(pdim * numNodes, pdim * numNodes)
                Sgp = zeros(dim, dim)

                # energy path buffers (Gauss-level, reused)
                CgpM = @MMatrix zeros(3, 3)  # mutable accumulator

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    fill!(Ke, 0.0)

                    # fetch element nodal arrays once
                    Selem = stress !== nothing ? Smap[elem] : nothing
                    Celem = energy !== nothing ? Cmap[elem] : nothing

                    for k in eachindex(wip)
                        # --- invJ via StaticArrays (fast, no heap)
                        Jm = @SMatrix [
                            Jac[1, 3k-2] Jac[1, 3k-1] Jac[1, 3k];
                            Jac[2, 3k-2] Jac[2, 3k-1] Jac[2, 3k];
                            Jac[3, 3k-2] Jac[3, 3k-1] Jac[3, 3k]
                        ]
                        invJ = transpose(inv(Jm))
                        w = jacDet[k] * wip[k]

                        if stress !== nothing
                            # --- interpolate stress to Gauss
                            fill!(Sgp, 0.0)
                            @inbounds for a in 1:numNodes
                                Na = H[a, k]
                                base = 9a - 8
                                # manual reshape(Selem[...],3,3) without alloc
                                Sgp[1, 1] += Na * Selem[base+0, 1]
                                Sgp[2, 1] += Na * Selem[base+1, 1]
                                Sgp[3, 1] += Na * Selem[base+2, 1]
                                Sgp[1, 2] += Na * Selem[base+3, 1]
                                Sgp[2, 2] += Na * Selem[base+4, 1]
                                Sgp[3, 2] += Na * Selem[base+5, 1]
                                Sgp[1, 3] += Na * Selem[base+6, 1]
                                Sgp[2, 3] += Na * Selem[base+7, 1]
                                Sgp[3, 3] += Na * Selem[base+8, 1]
                            end
                        else
                            # --- interpolate C to Gauss (MMatrix accumulator)
                            fill!(CgpM, 0.0)
                            @inbounds for a in 1:numNodes
                                Na = H[a, k]
                                base = 9a - 8
                                CgpM[1, 1] += Na * Celem[base+0, 1]
                                CgpM[2, 1] += Na * Celem[base+1, 1]
                                CgpM[3, 1] += Na * Celem[base+2, 1]
                                CgpM[1, 2] += Na * Celem[base+3, 1]
                                CgpM[2, 2] += Na * Celem[base+4, 1]
                                CgpM[3, 2] += Na * Celem[base+5, 1]
                                CgpM[1, 3] += Na * Celem[base+6, 1]
                                CgpM[2, 3] += Na * Celem[base+7, 1]
                                CgpM[3, 3] += Na * Celem[base+8, 1]
                            end
                            # convert once, call stress_from_energy (expects SMatrix{3,3})
                            Cgp = SMatrix{3,3,Float64}(CgpM)
                            S = LowLevelFEM.stress_from_energy(energy, Cgp, params)  # should return 3×3 (SMatrix or Matrix)
                            Sgp .= S
                        end

                        # --- geometric stiffness contribution
                        @inbounds for a in 1:numNodes, b in 1:numNodes
                            # ∇N = invJ * grad_hat (manual for speed)
                            ghA = @view ∇h[3a-2:3a, k]
                            ghB = @view ∇h[3b-2:3b, k]

                            ∇Na1 = invJ[1, 1] * ghA[1] + invJ[1, 2] * ghA[2] + invJ[1, 3] * ghA[3]
                            ∇Na2 = invJ[2, 1] * ghA[1] + invJ[2, 2] * ghA[2] + invJ[2, 3] * ghA[3]
                            ∇Na3 = invJ[3, 1] * ghA[1] + invJ[3, 2] * ghA[2] + invJ[3, 3] * ghA[3]

                            ∇Nb1 = invJ[1, 1] * ghB[1] + invJ[1, 2] * ghB[2] + invJ[1, 3] * ghB[3]
                            ∇Nb2 = invJ[2, 1] * ghB[1] + invJ[2, 2] * ghB[2] + invJ[2, 3] * ghB[3]
                            ∇Nb3 = invJ[3, 1] * ghB[1] + invJ[3, 2] * ghB[2] + invJ[3, 3] * ghB[3]

                            # t = Sgp * ∇Nb
                            t1 = Sgp[1, 1] * ∇Nb1 + Sgp[1, 2] * ∇Nb2 + Sgp[1, 3] * ∇Nb3
                            t2 = Sgp[2, 1] * ∇Nb1 + Sgp[2, 2] * ∇Nb2 + Sgp[2, 3] * ∇Nb3
                            t3 = Sgp[3, 1] * ∇Nb1 + Sgp[3, 2] * ∇Nb2 + Sgp[3, 3] * ∇Nb3

                            g = ∇Na1 * t1 + ∇Na2 * t2 + ∇Na3 * t3

                            # distribute to displacement components (i=i only)
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + i
                                Ke[ia, ib] += g * w
                            end
                        end
                    end

                    # --- scatter (pos-style)
                    @inbounds for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


initialStressMatrix1 (generic function with 1 method)

In [208]:
function internalForceVector0(problem::Problem; P::Union{Nothing,TensorField}=nothing,
    F::Union{Nothing,TensorField}=nothing,
    energy::Union{Nothing,Function}=nothing,
    params=nothing)

    gmsh.model.setCurrent(problem.name)

    @assert xor(P !== nothing, energy !== nothing) """
    Either provide `P` (TensorField),
    or provide (`energy`, `params`, `F`) to compute it at Gauss points.
    """

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    if P !== nothing
        Pe = nodesToElements(P)
        Pmap = Dict(zip(Pe.numElem, Pe.A))
    end

    if F !== nothing
        Fe = nodesToElements(F)
        Fmap = Dict(zip(Fe.numElem, Fe.A))
    end

    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of P)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    if P !== nothing
                        Pelem = Pmap[elem]
                    end
                    if F !== nothing
                        Felem = Fmap[elem]
                    end
                    fe = zeros(pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- interpolate P to Gauss point using shape functions
                        #Pgp = zeros(dim, dim)
                        #for a in 1:numNodes
                        #    Na = H[a, k]
                        #    Pgp .+= Na * reshape(Pelem[9a-8:9a, 1], dim, dim)
                        #end

                        if P !== nothing
                            # -----------------------------
                            # OLD PATH: interpolate given P
                            # -----------------------------
                            Pgp = zeros(dim, dim)
                            for a in 1:numNodes
                                Na = H[a, k]
                                Pgp .+= Na * reshape(Pelem[9a-8:9a, 1], dim, dim)
                            end
                        else
                            # --------------------------------
                            # NEW PATH: energy-based P
                            # --------------------------------
                            # F at Gauss (már úgyis kiszámolod!)
                            Fgp = zeros(dim, dim)
                            for a in 1:numNodes
                                Na = H[a, k]
                                Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                            end
                            Fgpm = @SMatrix [
                                Fgp[1, 1] Fgp[1, 2] Fgp[1, 3];
                                Fgp[2, 1] Fgp[2, 2] Fgp[2, 3];
                                Fgp[3, 1] Fgp[3, 2] Fgp[3, 3]
                            ]

                            # Right Cauchy–Green
                            Cmat = transpose(Fgpm) * Fgpm

                            # PK2 stress from energy
                            S = LowLevelFEM.stress_from_energy(energy, Cmat, params)

                            # Convert PK2 → PK1:  P = F * S
                            Pgp = Matrix(Fgpm * S)
                        end


                        for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                fe[ia] += dot(Pgp[i, :], ∇Na) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:numNodes, i in 1:dim
                        Ig = (nodeTags[a] - 1) * pdim + i
                        f[Ig] += fe[(a-1)*pdim+i]
                    end
                end
            end
        end
    end

    if pdim ≠ 3
        error("dim ≠ 3 is not implemented yet")
    end
    return VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end

internalForceVector0 (generic function with 1 method)

In [209]:
function internalForceVector1(
    problem::Problem;
    P::Union{Nothing,TensorField}=nothing,
    F::Union{Nothing,TensorField}=nothing,
    energy::Union{Nothing,Function}=nothing,
    params=nothing
)
    gmsh.model.setCurrent(problem.name)

    @assert xor(P !== nothing, energy !== nothing) """
    Either provide `P` (TensorField),
    or provide (`energy`, `params`, `F`) to compute it at Gauss points.
    """

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non
    @assert dim == 3 && pdim == 3  # jelen implementáció

    # --- elementwise maps
    Pmap = nothing
    Fmap = nothing
    if P !== nothing
        Pe = nodesToElements(P)
        Pmap = Dict(zip(Pe.numElem, Pe.A))
    else
        @assert F !== nothing
        Fe = nodesToElements(F)
        Fmap = Dict(zip(Fe.numElem, Fe.A))
    end

    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss$(2order + 1)")

                # --- shape functions
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- grad shape
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                # --- reusable buffers
                fe = zeros(pdim * numNodes)
                Pgp = zeros(3, 3)  # Float64 buffer
                Fgp = zeros(3, 3)  # only used in energy-path

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Pelem = (P !== nothing) ? Pmap[elem] : nothing
                    Felem = (energy !== nothing) ? Fmap[elem] : nothing

                    fill!(fe, 0.0)

                    for k in eachindex(wip)
                        # --- invJ via StaticArrays
                        Jm = @SMatrix [
                            Jac[1, 3k-2] Jac[1, 3k-1] Jac[1, 3k];
                            Jac[2, 3k-2] Jac[2, 3k-1] Jac[2, 3k];
                            Jac[3, 3k-2] Jac[3, 3k-1] Jac[3, 3k]
                        ]
                        invJ = transpose(inv(Jm))
                        w = jacDet[k] * wip[k]

                        if P !== nothing
                            # --- interpolate P to Gauss into Pgp buffer (no alloc)
                            fill!(Pgp, 0.0)
                            @inbounds for a in 1:numNodes
                                Na = H[a, k]
                                base = 9a - 8
                                # column-major (ugyanaz, mint reshape(x,3,3))
                                Pgp[1, 1] += Na * Pelem[base+0, 1]
                                Pgp[2, 1] += Na * Pelem[base+1, 1]
                                Pgp[3, 1] += Na * Pelem[base+2, 1]

                                Pgp[1, 2] += Na * Pelem[base+3, 1]
                                Pgp[2, 2] += Na * Pelem[base+4, 1]
                                Pgp[3, 2] += Na * Pelem[base+5, 1]

                                Pgp[1, 3] += Na * Pelem[base+6, 1]
                                Pgp[2, 3] += Na * Pelem[base+7, 1]
                                Pgp[3, 3] += Na * Pelem[base+8, 1]
                            end
                        else
                            # --- energy path: Fgp -> C -> S -> P = F*S
                            fill!(Fgp, 0.0)
                            @inbounds for a in 1:numNodes
                                Na = H[a, k]
                                base = 9a - 8
                                Fgp[1, 1] += Na * Felem[base+0, 1]
                                Fgp[2, 1] += Na * Felem[base+1, 1]
                                Fgp[3, 1] += Na * Felem[base+2, 1]
                                Fgp[1, 2] += Na * Felem[base+3, 1]
                                Fgp[2, 2] += Na * Felem[base+4, 1]
                                Fgp[3, 2] += Na * Felem[base+5, 1]
                                Fgp[1, 3] += Na * Felem[base+6, 1]
                                Fgp[2, 3] += Na * Felem[base+7, 1]
                                Fgp[3, 3] += Na * Felem[base+8, 1]
                            end

                            Fm = SMatrix{3,3,Float64}(Fgp)
                            Cmat = transpose(Fm) * Fm
                            S = LowLevelFEM.stress_from_energy(energy, Cmat, params)  # 3×3 (SMatrix ok)
                            Pm = Fm * S

                            # copy back into Pgp buffer (avoid Matrix(Pm))
                            @inbounds begin
                                Pgp[1, 1] = Pm[1, 1]
                                Pgp[1, 2] = Pm[1, 2]
                                Pgp[1, 3] = Pm[1, 3]
                                Pgp[2, 1] = Pm[2, 1]
                                Pgp[2, 2] = Pm[2, 2]
                                Pgp[2, 3] = Pm[2, 3]
                                Pgp[3, 1] = Pm[3, 1]
                                Pgp[3, 2] = Pm[3, 2]
                                Pgp[3, 3] = Pm[3, 3]
                            end
                        end

                        # --- fe accumulation
                        @inbounds for a in 1:numNodes
                            gh = @view ∇h[3a-2:3a, k]
                            ∇Na1 = invJ[1, 1] * gh[1] + invJ[1, 2] * gh[2] + invJ[1, 3] * gh[3]
                            ∇Na2 = invJ[2, 1] * gh[1] + invJ[2, 2] * gh[2] + invJ[2, 3] * gh[3]
                            ∇Na3 = invJ[3, 1] * gh[1] + invJ[3, 2] * gh[2] + invJ[3, 3] * gh[3]

                            base = (a - 1) * pdim
                            # dot(Pgp[i,:], ∇Na) kézzel, hogy ne allokáljon view/vec
                            fe[base+1] += (Pgp[1, 1] * ∇Na1 + Pgp[1, 2] * ∇Na2 + Pgp[1, 3] * ∇Na3) * w
                            fe[base+2] += (Pgp[2, 1] * ∇Na1 + Pgp[2, 2] * ∇Na2 + Pgp[2, 3] * ∇Na3) * w
                            fe[base+3] += (Pgp[3, 1] * ∇Na1 + Pgp[3, 2] * ∇Na2 + Pgp[3, 3] * ∇Na3) * w
                        end
                    end

                    # scatter fe into global f
                    @inbounds for a in 1:numNodes
                        nt = nodeTags[a]
                        base = (a - 1) * pdim
                        f[(nt-1)*pdim+1] += fe[base+1]
                        f[(nt-1)*pdim+2] += fe[base+2]
                        f[(nt-1)*pdim+3] += fe[base+3]
                    end
                end
            end
        end
    end

    return VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end


internalForceVector1 (generic function with 1 method)

In [210]:
@inline function _tangent_load_helper(f, h, x, y, z, nnet, j, l, nsteps)
    if f isa Number
        return fill(f, nsteps)
    elseif f isa Function
        return fill(f(x, y, z), nsteps)
    elseif f isa ScalarField && isNodal(f)
        v = h[:, j]' * f.a[nnet[l, :], :]
        return vec(v)
    else
        error("externalTangentFollowerTL: invalid load type.")
    end
end

_tangent_load_helper (generic function with 1 method)

In [211]:
function externalTangentFollower1(
    problem::Problem,
    loads::Vector{BoundaryCondition};
    F::TensorField)

    gmsh.model.setCurrent(problem.name)

    dim  = problem.dim
    pdim = problem.pdim
    non  = problem.non
    dof  = pdim * non
    @assert dim == 3 && pdim == 3

    # --- elementwise F
    Fe   = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    #Id = Matrix{Float64}(I, 3, 3)
    Id = [1.0 0 0; 0 1 0; 0 0 1]

    # --- pos-style sparse
    len = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, len)
    J = Vector{Int}(undef, len)
    V = Vector{Float64}(undef, len)
    pos = 1

    # --- global node coordinates cache
    ncoord2 = zeros(3 * non)

    for bc in loads
        name = bc.phName

        fx = bc.fx
        fy = bc.fy
        fz = bc.fz
        p  = bc.p

        fx = (fx isa ScalarField && isElementwise(fx)) ? elementsToNodes(fx) : fx
        fy = (fy isa ScalarField && isElementwise(fy)) ? elementsToNodes(fy) : fy
        fz = (fz isa ScalarField && isElementwise(fz)) ? elementsToNodes(fz) : fz
        p  = (p  isa ScalarField && isElementwise(p )) ? elementsToNodes(p ) : p

        if p !== nothing && (fx !== nothing || fy !== nothing || fz !== nothing)
            error("externalTangentFollowerTL: p and fx/fy/fz cannot be defined together.")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(name)

        for (edim, etag) in dimTags
            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            nodeTagsAll, ncoord, _ =
                gmsh.model.mesh.getNodes(edim, etag, true, false)

            @inbounds begin
                ncoord2[nodeTagsAll*3 .- 2] = ncoord[1:3:end]
                ncoord2[nodeTagsAll*3 .- 1] = ncoord[2:3:end]
                ncoord2[nodeTagsAll*3 .- 0] = ncoord[3:3:end]
            end

            nv = (p !== nothing) ? -normalVector(problem, name) : nothing

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss$(2order + 1)")

                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                h = reshape(hfun, numNodes, :)

                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                ndofe = pdim * numNodes

                # --- reusable buffers
                Ke   = zeros(ndofe, ndofe)
                Fgp  = zeros(3, 3)
                tgp  = zeros(3)
                dt0  = zeros(3)
                tloc = zeros(3)

                for (l, elem) in enumerate(elemTags[it])
                    nodeTags =
                        @view elemNodeTags[it][(l-1)*numNodes+1:l*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Fnode = Fmap[elem]
                    fill!(Ke, 0.0)

                    for j in eachindex(wip)
                        Jm = @SMatrix [
                            Jac[1,3j-2] Jac[1,3j-1] Jac[1,3j];
                            Jac[2,3j-2] Jac[2,3j-1] Jac[2,3j];
                            Jac[3,3j-2] Jac[3,3j-1] Jac[3,3j]
                        ]
                        invJ = transpose(inv(Jm))
                        w = jacDet[j] * wip[j]

                        # --- F at Gauss (reshape-safe)
                        fill!(Fgp, 0.0)
                        @inbounds for a in 1:numNodes
                            Na = h[a, j]
                            base = 9a - 8
                            Fgp[1,1] += Na * Fnode[base+0,1]
                            Fgp[2,1] += Na * Fnode[base+1,1]
                            Fgp[3,1] += Na * Fnode[base+2,1]
                            Fgp[1,2] += Na * Fnode[base+3,1]
                            Fgp[2,2] += Na * Fnode[base+4,1]
                            Fgp[3,2] += Na * Fnode[base+5,1]
                            Fgp[1,3] += Na * Fnode[base+6,1]
                            Fgp[2,3] += Na * Fnode[base+7,1]
                            Fgp[3,3] += Na * Fnode[base+8,1]
                        end

                        Fm = SMatrix{3,3,Float64}(Fgp)
                        Finv = inv(Fm)
                        FinvT = transpose(Finv)
                        Jgp = det(Fm)

                        # --- Gauss coordinates
                        x = 0.0; y = 0.0; z = 0.0
                        @inbounds for a in 1:numNodes
                            Na = h[a,j]
                            nt = nodeTags[a]
                            x += Na * ncoord2[3nt-2]
                            y += Na * ncoord2[3nt-1]
                            z += Na * ncoord2[3nt-0]
                        end

                        # --- traction
                        fill!(tgp, 0.0)
                        if p !== nothing
                            pval = _tangent_load_helper(p, h, x, y, z, nodeTags, j, 1, 1)[1]
                            for d in 1:3
                                tgp[d] = -pval * (h[:,j]' * nv[d].a[nodeTags,1])
                            end
                        else
                            if fx !== nothing
                                tgp[1] = _tangent_load_helper(fx, h, x, y, z, nodeTags, j, 1, 1)[1]
                            end
                            if fy !== nothing
                                tgp[2] = _tangent_load_helper(fy, h, x, y, z, nodeTags, j, 1, 1)[1]
                            end
                            if fz !== nothing
                                tgp[3] = _tangent_load_helper(fz, h, x, y, z, nodeTags, j, 1, 1)[1]
                            end
                        end

                        tgp .= Fgp * tgp
                        tloc .= Finv * tgp

                        for a in 1:numNodes, b in 1:numNodes
                            ghb = @view ∇h[3b-2:3b, j]
                            ∇Nb = invJ * ghb

                            for jb in 1:3
                                # --- unchanged physics
                                fcol = Finv[:,jb]
                                α = dot(fcol, ∇Nb)

                                dJFmT = Jgp * FinvT * (α*Id - (∇Nb*fcol'))
                                dt0 .= dJFmT * tgp

                                dFt = dot(∇Nb, tloc)
                                @inbounds for ia in 1:3
                                    dt0[ia] += Jgp * FinvT[ia,jb] * dFt
                                end

                                @inbounds for ia in 1:3
                                    Ke[(a-1)*3+ia,(b-1)*3+jb] +=
                                        h[a,j] * dt0[ia] * w
                                end
                            end
                        end
                    end

                    @inbounds for a in 1:ndofe, b in 1:ndofe
                        I[pos] = (nodeTags[div(a-1,3)+1]-1)*3 + mod(a-1,3) + 1
                        J[pos] = (nodeTags[div(b-1,3)+1]-1)*3 + mod(b-1,3) + 1
                        V[pos] = Ke[a,b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I,pos-1); resize!(J,pos-1); resize!(V,pos-1)
    K = sparse(I,J,V,dof,dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


externalTangentFollower1 (generic function with 1 method)

In [212]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube", E=2e2)
prob = Problem([mat], type=:VectorField)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00100698s, CPU 0.001015s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000826187s, CPU 0.000455s)
Info    : Meshing 3D.

Problem("cube1", :VectorField, 3, 3, Material[Material("cube", :Hooke, 200.0, 0.3, 115.38461538461537, 76.92307692307692, 166.66666666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 6859, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [213]:
F = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1]);

In [214]:
#Fleft = nodesToElements(elementsToNodes(F), onPhysicalGroup="left")

#Kext = externalTangentFollowerTL(prob; F=Fleft, traction_phName="left", t_spatial=[1, 0, 0])

In [215]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)


load = BoundaryCondition("right", fy=(x, y, z) -> -(z - 0.5) * 30, fz=(x, y, z) -> (y - 0.5) * 30)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, var"#72#73"(), var"#74#75"(), nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [216]:
μ = mat.μ
λ = mat.λ

115.38461538461537

In [217]:
f_ext0 = loadVector(prob, [load])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [218]:
showDoFResults(f_ext0, name="f")

0

In [219]:
u = vectorField(prob, "cube", [0, 0, 0])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [220]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [221]:
I = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [222]:
P = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])
S = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])

nodal TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [223]:
λ0 = λ
μ0 = μ
C1 = [λ0+2μ0 λ0 λ0 0 0 0;
    λ0 λ0+2μ0 λ0 0 0 0;
    λ0 λ0 λ0+2μ0 0 0 0;
    0 0 0 μ0 0 0;
    0 0 0 0 μ0 0;
    0 0 0 0 0 μ0]

6×6 Matrix{Float64}:
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [224]:
function svk_energy2(C, p)
    λ = p.λ
    μ = p.μ

    I = [1.0 0 0; 0 1 0; 0 0 1]

    E = (C - I) / 2

    return (λ * tr(E)^2 + 2μ * dot(E, E)) / 2
end


svk_energy2 (generic function with 1 method)

In [225]:
using StaticArrays

Cst = SMatrix{3,3}([1.0 0 0; 0 1 0; 0 0 1])
LowLevelFEM._tangent_from_energy(svk_energy2, Cst, (λ=λ, μ=μ))

6×6 SMatrix{6, 6, Float64, 36} with indices SOneTo(6)×SOneTo(6):
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [226]:
C1

6×6 Matrix{Float64}:
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [227]:
using StaticArrays

m = [1.0 0 1; 0 1 0; 0 0 1]
c = m' * m
Cst = SMatrix{3,3}(c)
LowLevelFEM._stress_from_energy(svk_energy2, Cst, (λ=λ, μ=μ))

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 57.6923   0.0      76.9231
  0.0     57.6923    0.0
 76.9231   0.0     134.615

In [228]:
F0 = m
I0 = [1.0 0 0; 0 1 0; 0 0 1]
E0 = (F0' * F0 - I0) / 2
S0 = λ * tr(E0) * I0 + 2μ * E0
#P = F * S

3×3 Matrix{Float64}:
 57.6923   0.0      76.9231
  0.0     57.6923    0.0
 76.9231   0.0     134.615

In [229]:
Kmat = materialTangentMatrix(prob, F=F, energy=svk_energy2, params=(λ=λ, μ=μ))

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.214941437163648, 0.71225071225071, -0.7122507122507103, -0.4108895220006324, -0.1780626780626775, -0.047483380816712564, -2.2171573282684283, 0.35612535612535806, 0.18993352326685406, -0.41088952200063555  …  -1.0940170940171552, 1.5265566588595902e-15, -1.9761969838327786e-14, -7.94175371953127, -1.6181500583911657e-14, 1.3600232051658168e-14, 72.28616650838897, 1.955380302121057e-14, 9.547918011776346e-15, 584.7723963279515], 20577, 20577)

In [230]:
C = F' * F
Kgeo = initialStressMatrix(prob, energy=svk_energy2, params=(λ=λ, μ=μ), C=C)

sparse([1, 46, 73, 76, 100, 484, 508, 805, 1375, 1453  …  5832, 5838, 7374, 19599, 19695, 19707, 20511, 20523, 20571, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [-6.0577639582181085e-15, -3.2264621685491715e-16, 2.1425221077808206e-15, -4.943907760358248e-16, 2.8019819880610902e-15, 1.0119077500936618e-15, -3.300067713983044e-15, 5.263760606888137e-16, -1.1791228153415202e-15, -1.6805973146625665e-15  …  -1.0092270983503295e-15, -7.984848858671263e-16, 8.291595734045311e-16, 1.0257400562934199e-15, -2.9370914099432145e-16, -3.7275725515487486e-16, -2.9370914099433023e-16, -3.727572551548858e-16, -5.284330225342807e-15, -4.571255734862617e-15], 20577, 20577)

In [231]:
f_int = internalForceVector(prob, energy=svk_energy2, params=(λ=λ, μ=μ), F=F)

nodal VectorField
[4.202952757135736e-17; 3.929882386451069e-16; … ; 1.4563753103182749e-16; 1.4563753103182547e-16;;]

In [232]:
Kmat0 = materialTangentMatrix(prob, F=F, C=C1)

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.214941437163648, 0.71225071225071, -0.7122507122507103, -0.4108895220006324, -0.1780626780626775, -0.047483380816712564, -2.2171573282684283, 0.35612535612535806, 0.18993352326685406, -0.41088952200063555  …  -1.0940170940171552, 1.5265566588595902e-15, -1.9761969838327786e-14, -7.94175371953127, -1.6181500583911657e-14, 1.3600232051658168e-14, 72.28616650838897, 1.955380302121057e-14, 9.547918011776346e-15, 584.7723963279515], 20577, 20577)

In [233]:
Kmat0 = materialTangentMatrix0(prob, F=F, energy=svk_energy2, params=(λ=λ, μ=μ))

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.214941437163648, 0.71225071225071, -0.7122507122507103, -0.4108895220006324, -0.1780626780626775, -0.047483380816712564, -2.2171573282684283, 0.35612535612535806, 0.18993352326685406, -0.41088952200063555  …  -1.0940170940171552, 1.5265566588595902e-15, -1.9761969838327786e-14, -7.94175371953127, -1.6181500583911657e-14, 1.3600232051658168e-14, 72.28616650838897, 1.955380302121057e-14, 9.547918011776346e-15, 584.7723963279515], 20577, 20577)

In [234]:
Kgeo0 = initialStressMatrix(prob, stress=S)

sparse(Int64[], Int64[], Float64[], 20577, 20577)

In [235]:
C = F' * F
Kgeo0 = initialStressMatrix0(prob, energy=svk_energy2, params=(λ=λ, μ=μ), C=C)

sparse([1, 46, 73, 76, 100, 484, 508, 805, 1375, 1453  …  5832, 5838, 7374, 19599, 19695, 19707, 20511, 20523, 20571, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [-6.0577639582181085e-15, -3.2264621685491715e-16, 2.1425221077808206e-15, -4.943907760358248e-16, 2.8019819880610902e-15, 1.0119077500936618e-15, -3.300067713983044e-15, 5.263760606888137e-16, -1.1791228153415202e-15, -1.6805973146625665e-15  …  -1.0092270983503295e-15, -7.984848858671263e-16, 8.291595734045311e-16, 1.0257400562934199e-15, -2.9370914099432145e-16, -3.7275725515487486e-16, -2.9370914099433023e-16, -3.727572551548858e-16, -5.284330225342807e-15, -4.571255734862617e-15], 20577, 20577)

In [236]:
f_int0 = internalForceVector(prob, P=P)

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [237]:
f_int0 = internalForceVector0(prob, energy=svk_energy2, params=(λ=λ, μ=μ), F=F)

nodal VectorField
[4.202952757135736e-17; 3.929882386451069e-16; … ; 1.4563753103182749e-16; 1.4563753103182547e-16;;]

In [238]:
norm(Kmat0.A - Kmat.A)

0.0

In [239]:
norm(Kgeo0.A - Kgeo.A)

0.0

In [240]:
norm(f_int0.a - f_int.a)

0.0

In [241]:
#showDoFResults(f_int0, name="f_int_P")
#showDoFResults(f_int, name="f_int_ψ")
#openPostProcessor()

## Innen indul az iteráció

In [242]:
for i in 1:10

    Kmat = materialTangentMatrix1(prob, F=F, energy=svk_energy2, params=(λ=λ, μ=μ))

    C = F' * F
    Kgeo = initialStressMatrix1(prob, energy=svk_energy2, params=(λ=λ, μ=μ), C=C)

    Kint = Kmat + Kgeo
    Fright = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")

    Kext = externalTangentFollower1(prob, [load], F=Fright)
    f_int = internalForceVector1(prob, energy=svk_energy2, params=(λ=λ, μ=μ), F=F)
    K = Kint - Kext
    FF = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")
    f_ext = loadVector(prob, [load], F=FF)
    Δu = solveField(
        K,
        f_ext - f_int;
        support=[suppX]
    )

    fixed = constrainedDoFs(prob, [suppX])
    free = freeDoFs(prob, [suppX])
    #norm(Δu.a[:, 1])
    r = f_ext - f_int
    println("‖r_free‖  = ", norm(r.a[free, 1]))
    println("‖r_fixed‖ = ", norm(r.a[fixed, 1]))
    println("‖r‖ = ", norm(r.a[:, 1]))
    println("‖Δu‖ = ", norm(Δu.a[:, 1]))

    u += Δu
    showDoFResults(u)
    F = I + u ∘ ∇
    E = 0.5 * (F' * F - I)
    S = λ * trace(E) * I + 2μ * E
    P = F * S
    #P = λ * trace(F - I) * I + 2μ * (F - I)

    if norm(Δu.a[:, 1]) < 0.01
        break
    end

end

‖r_free‖  = 0.7372768252552513
‖r_fixed‖ = 9.20414306661775e-15
‖r‖ = 0.7372768252552513
‖Δu‖ = 54.215657539328745
‖r_free‖  = 1697.878718822686
‖r_fixed‖ = 0.590698713409716
‖r‖ = 1697.878821575891
‖Δu‖ = 34.30032456589198
‖r_free‖  = 468.50848494373986
‖r_fixed‖ = 0.8117542546374324
‖r‖ = 468.5091881801769
‖Δu‖ = 10.533554121321696
‖r_free‖  = 117.25625812021656
‖r_fixed‖ = 0.5389526695105925
‖r‖ = 117.25749672551783
‖Δu‖ = 21.403047231673817
‖r_free‖  = 25.400706771429213
‖r_fixed‖ = 0.4450954532262171
‖r‖ = 25.404606166020614
‖Δu‖ = 6.905303491944291
‖r_free‖  = 4.028526232078633
‖r_fixed‖ = 0.38516749040081594
‖r‖ = 4.04689728041216
‖Δu‖ = 2.1350955715924713
‖r_free‖  = 1.3496211809132925
‖r_fixed‖ = 0.3551173319051405
‖r‖ = 1.3955592611527523
‖Δu‖ = 3.2845431518334016
‖r_free‖  = 3.717581115263214
‖r_fixed‖ = 0.34657039676790424
‖r‖ = 3.733700629198536
‖Δu‖ = 1.846102998283744
‖r_free‖  = 0.8547046670316917
‖r_fixed‖ = 0.35476358898043503
‖r‖ = 0.9254065441264364
‖Δu‖ = 4.6356559

In [243]:
openPostProcessor()

XRequest.18: BadValue 0x0


-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [244]:
Kgeo.A

20577×20577 SparseMatrixCSC{Float64, Int64} with 1167051 stored entries:
⎡⣿⣿⣿⡿⢺⡧⣿⠼⣿⣾⣟⣿⣟⣛⣿⣶⣾⣷⣤⣜⣓⣀⣘⣓⣀⣐⣓⣀⣐⣓⣀⣐⣓⣀⣐⣟⣛⣻⣟⣿⎤
⎢⣿⡿⠻⣦⣼⡇⣿⢠⣤⣴⡶⣯⣽⣷⣯⣭⣭⣭⣭⣭⣭⣭⣭⣽⣿⣿⣿⡿⠿⠿⠷⠶⠶⠶⢶⡶⠶⣶⡶⣦⎥
⎢⠾⡶⠶⠿⢿⣷⡟⠘⣷⢰⡖⣶⣿⣿⣷⠀⠀⣶⠀⠀⣾⠉⠉⣿⠉⠉⣯⠉⠉⡯⠙⠛⡿⠛⠛⡟⠛⠓⡖⠒⎥
⎢⣛⡟⠛⣛⣛⠉⢿⣷⡏⢸⡇⣿⣿⣿⡏⠀⢸⡏⠀⢸⡏⠀⢸⡏⠀⢸⡏⠀⢸⡏⠀⢸⡏⠀⢸⡇⠀⢸⡇⢶⎥
⎢⣻⣿⢀⣿⢙⣛⣋⣉⡿⣯⣁⢻⠉⠉⡛⠿⣭⡅⠀⠀⠁⠀⠀⠁⠀⠈⠁⠀⠈⠁⠀⠈⠁⠀⠈⣁⣀⡈⣁⡈⎥
⎢⣿⣽⡼⣯⢸⣭⣭⣭⣥⣘⠿⣧⡀⠀⣧⣄⣀⣤⣄⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠻⠯⠿⠯⎥
⎢⣿⢹⢷⣿⣿⣿⣿⣿⡇⠀⠀⠈⠻⣦⡀⠉⠉⠉⠉⠛⠛⠛⠛⠿⠿⠿⠶⣶⣶⣶⣦⣤⣤⣤⣄⣀⣀⣀⠀⠀⎥
⎢⢻⣿⡏⣿⠙⠛⠋⠉⣿⡌⠉⢿⡄⠈⠻⣦⡀⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠉⠉⠉⠉⎥
⎢⢾⣿⡇⣿⢠⣤⡶⠶⠇⠿⠀⣼⡇⠀⣤⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣀⢿⡇⣿⠀⠀⣀⣀⠀⠀⠀⢹⣧⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⢸⡇⣿⡞⠛⠋⠉⠁⠀⠀⢸⣿⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢶⢸⣇⣿⣧⣤⡶⠶⠄⠀⠀⠈⣿⡄⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢀⢸⣿⣿⡇⠀⣀⣀⡀⠀⠀⠀⣿⡇⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⢸⣿⡿⡏⠛⠋⠉⠁⠀⠀⠀⢸⣧⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢴⢸⣿⡇⡧⡤⡶⠶⠆⠀⠀⠀⢸⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⎥
⎢⢀⢸⢹⡇⣷⠀⣀⣀⡀⠀⠀⠀⠈⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⎥
⎢⠙⢸⢸⡇⣿⠋⠋⠉⠁⠀⠀⠀⠀⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⎥
⎢⣴⢼⢸⡷⣿⠤⠶⠶⠆⢠⣄⠀⠀⢹⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⠀⎥
⎢⣿⣸⢸⣧⢿⠀⣀⣀⡀⠸⡿⡆⠀⢸⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡹⣧⎥
⎣⣿⣽⠸⣯⢸⠉⢩⣍⡁⠸⡿⡇⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⠷⣮⡻⣮⎦

In [245]:
gmsh.finalize()